In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from src.preprocessing import preprocess_data, calculate_daily_returns
from src.backtesting import run_backtest, calculate_performance_metrics, plot_backtest_results

plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
RAW_DATA_PATH = 'data/raw/financial_data.csv'
raw_data = pd.read_csv(RAW_DATA_PATH, header=[0, 1], index_col=0, parse_dates=True)
adj_close_prices = preprocess_data(raw_data)
daily_returns = calculate_daily_returns(adj_close_prices)

# Load the recommended weights from the previous notebook
%store -r recommended_weights
print("Loaded Recommended Strategy Weights:")
print(recommended_weights)

In [ ]:
backtest_start_date = '2024-08-01'
backtest_end_date = '2025-07-31'
backtest_returns = daily_returns.loc[backtest_start_date:backtest_end_date]

# Define benchmark weights (60% SPY, 40% BND)
benchmark_weights = {'SPY': 0.60, 'BND': 0.40, 'TSLA': 0.0}

In [ ]:
backtest_results = run_backtest(backtest_returns, recommended_weights, benchmark_weights)

print("\nBacktest Results (Final Portfolio Value):")
print(backtest_results.iloc[-1])

In [ ]:
FIGURE_PATH = 'reports/figures/backtest_performance.png'
plot_backtest_results(backtest_results, save_path=FIGURE_PATH)

In [ ]:
strategy_daily_returns = backtest_returns.dot(pd.Series(recommended_weights).reindex(backtest_returns.columns).fillna(0))
benchmark_daily_returns = backtest_returns.dot(pd.Series(benchmark_weights).reindex(backtest_returns.columns).fillna(0))

strategy_metrics = calculate_performance_metrics(backtest_results['Strategy'], strategy_daily_returns)
benchmark_metrics = calculate_performance_metrics(backtest_results['Benchmark'], benchmark_daily_returns)

performance_df = pd.DataFrame({
    'Strategy': strategy_metrics,
    'Benchmark (60/40)': benchmark_metrics
})

print("\n--- Performance Metrics Comparison ---")
print(performance_df)